In [3]:
import cv2
from PIL import Image, ImageEnhance

In [4]:
import import_ipynb
import NeuralNetwork4 as neural
import CharacterSeparation2 as cs

importing Jupyter notebook from NeuralNetwork4.ipynb
importing Jupyter notebook from CharacterSeparation2.ipynb


In [5]:
def separateFraction(box,origWidth,origHeight,image):
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]
    boxH = h
        
    #denominator
    y = y + h
    h = origHeight - (origHeight - y)
    roi = image[y:y+h, x:x+w]
    cv2.imwrite("separated/denom.png", roi)
    cv2.imwrite("data/denom.png", roi)
    denom = reconstructExpression("denom.png")
    
    #numerator
    h = (origHeight-h) -boxH
    y = y - boxH - h
    roi = image[y:y+h, x:x+w]
    cv2.imwrite("separated/num.png", roi)
    cv2.imwrite("data/num.png", roi)
    num = reconstructExpression("num.png")
 
    #rest
    x = x + w
    w = origWidth-x
    h = origHeight
    roi = image[y:y+h, x:x+w]
    cv2.imwrite("separated/rest.png", roi)
    cv2.imwrite("data/rest.png", roi)
    rest = reconstructExpression("rest.png")
    
    returnString = "("+num+"/"+denom+")"+rest

    return returnString

In [6]:
def checkFractionBar(aR):
    if(aR>=5.00):
        return True

In [7]:
def checkSquareRoot(box,prevBox):
    if(prevBox[0]<=(box[0]+box[2])<=(prevBox[0]+prevBox[2])):
        if(prevBox[1]<=(box[1]+box[3])<=(prevBox[1]+prevBox[3])):
            return True

In [8]:
def checkExponent(centroid,prevCentroid):
    if(centroid[0]>prevCentroid[0]):
        if(centroid[1]<prevCentroid[1]-60):
            return True

In [9]:
def reconstructExpression(originalFile):
    boundingBox = []
    centroid = []
    fileName = []
    
    finalString = ""
    carryRoot = False
    carryExp = False
    charFound = False

    i = cs.separateChars(originalFile)
    
    boundingBox = cs.getBoundingBoxes()
    centroid = cs.getCentroids()
    fileName = cs.getFilenames()
    
    #read in brightened image
    image = cv2.imread("data/bright.png")
    origWidth, origHeight, channels = image.shape
    
    #bubble sort
    for passNum in range(0,i):
        for c in range(0, i-passNum-1):
            if(boundingBox[c][0]>boundingBox[c+1][0]):
                temp = boundingBox[c]
                boundingBox[c] = boundingBox[c+1]
                boundingBox[c+1] = temp
                temp2 = centroid[c]
                centroid[c] = centroid[c+1]
                centroid[c+1] = temp2
                temp3 = fileName[c]
                fileName[c] = fileName[c+1]
                fileName[c+1] = temp3
    
    for n in range(0,i):
        charFound = False
        aspectRatio = (boundingBox[n][2]/boundingBox[n][3])
        
        #square roots
        if(n>0):
            if(checkSquareRoot(boundingBox[n],boundingBox[n-1])==True):
                finalString = finalString + "sqrt("
                charFound = True
                carryRoot=True
                carryRootBox=n-1
                
            #exponents
            elif(checkExponent(centroid[n],centroid[n-1])==True):
                finalString = finalString + "^("
                charFound = True
                carryExp=True
                carryExpBox=n-1
        
        #fractions
        if(checkFractionBar(aspectRatio)==True):
            finalString = finalString + separateFraction(boundingBox[n],origWidth,origHeight,image)
            charFound = True
            break
        
        #carry check for roots
        if(carryRoot==True and carryExp==False):
            if(checkSquareRoot(boundingBox[n],boundingBox[carryRootBox])==True):
                finalString = finalString +str(neural.identifyChar(fileName))
                if(n==i-1):
                    finalString = finalString + ")"
                    carryRoot=False
            else:
                finalString = finalString + ")"
                carryRoot=False
                
        #carry check for exponents
        if(carryExp==True):
            if(checkExponent(centroid[n],centroid[carryExpBox])==True):
                finalString = finalString +str(neural.identifyChar(fileName))
                if(n==i-1):
                    finalString = finalString + ")"
                    carryExp=False
            else:
                finalString = finalString + ")"
                carryExp=False
        
        if(charFound == False):
            finalString = finalString +str(neural.identifyChar(fileName[n]))
            
        print(finalString)
    return finalString 


In [10]:
#def main():
 #   neural.setupNetwork(2025, 550, 11, 0.37, 100)
       #expression = reconstructExpression("num.png")
    #print(expression)
    #cv2.waitKey(0)
    

In [11]:
#if __name__ == "__main__":
 #   main()